<a href="https://colab.research.google.com/github/amrahmani/Marketing/blob/main/AIMarketing_Ch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/kaggle
!mv kaggle.json ~/kaggle/
!chmod 600 ~/kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/kaggle/kaggle.json': No such file or directory


In [37]:
!ls -al ~/.kaggle/kaggle.json

-rw------- 1 root root 73 Apr 27 06:55 /root/.kaggle/kaggle.json


In [28]:
!kaggle datasets download -d vijayuv/onlineretail -p ./data/

Dataset URL: https://www.kaggle.com/datasets/vijayuv/onlineretail
License(s): CC0-1.0
 69% 5.00M/7.20M [00:00<00:00, 9.65MB/s]
100% 7.20M/7.20M [00:00<00:00, 9.37MB/s]


In [36]:
!ls -al ./data/

total 51896
drwxr-xr-x 2 root root     4096 Apr 27 07:06 .
drwxr-xr-x 1 root root     4096 Apr 27 07:04 ..
-rw-r--r-- 1 root root 45580638 Sep 21  2019 OnlineRetail.csv
-rw-r--r-- 1 root root  7548702 Sep 21  2019 onlineretail.zip


In [31]:
!unzip ./data/onlineretail.zip -d ./data/

Archive:  ./data/onlineretail.zip
  inflating: ./data/OnlineRetail.csv  


In [38]:
import pandas as pd
# Specify the encoding explicitly, for example 'latin1' or 'ISO-8859-1'
df = pd.read_csv('./data/OnlineRetail.csv', encoding='latin1')
# Now you can work with the DataFrame 'df'
print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  
